In [20]:
# https://github.com/SSQ/Coursera-UW-Machine-Learning-Clustering-Retrieval/blob/master/Week%206%20PA%201/6_hierarchical_clustering.py

# import sframe                           
import matplotlib.pyplot as plt       
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import normalize
get_ipython().magic(u'matplotlib inline')


# wiki = sframe.SFrame('people_wiki.gl/')
dataframe=pd.read_csv('tfidf_of_paras.csv')
X = dataframe.iloc[:,2:].values
X = np.nan_to_num(X)
m, feat = X.shape

In [20]:
# def load_sparse_csr(filename):
#     loader = np.load(filename)
#     data = loader['data']
#     indices = loader['indices']
#     indptr = loader['indptr']
#     shape = loader['shape']
#     return csr_matrix( (data, indices, indptr), shape)

# tf_idf = load_sparse_csr('people_wiki_tf_idf.npz')
# map_index_to_word = sframe.SFrame('people_wiki_map_index_to_word.gl/')
tf_idf = X

# To be consistent with the k-means assignment, let's normalize all vectors to have unit norm.
tf_idf = normalize(tf_idf)

In [22]:
def bipartition(cluster, maxiter=400, num_runs=4, seed=None):
    data_matrix = cluster['matrix']
    dataframe   = cluster['dataframe']
    kmeans_model = KMeans(n_clusters=2, max_iter=maxiter, n_init=num_runs, random_state=seed, n_jobs=1)
    kmeans_model.fit(data_matrix)
    centroids, cluster_assignment = kmeans_model.cluster_centers_, kmeans_model.labels_
    data_matrix_left_child, data_matrix_right_child = data_matrix[cluster_assignment==0], data_matrix[cluster_assignment==1]
#     cluster_assignment_sa = sframe.SArray(cluster_assignment) ##     AG
    cluster_assignment_sa = cluster_assignment                 ##     AG
    dataframe_left_child, dataframe_right_child = dataframe[cluster_assignment_sa==0], dataframe[cluster_assignment_sa==1]
        
    
    # Package relevant variables for the child clusters
    cluster_left_child  = {'matrix': data_matrix_left_child,
                           'dataframe': dataframe_left_child,
                           'centroid': centroids[0]}
    cluster_right_child = {'matrix': data_matrix_right_child,
                           'dataframe': dataframe_right_child,
                           'centroid': centroids[1]}
    
    return (cluster_left_child, cluster_right_child)



# Note. For the purpose of the assignment, we set an explicit seed (`seed=1`) to produce identical 
# outputs for every run. In pratical applications, you might want to use different random seeds for all runs.
wiki_data = {'matrix': tf_idf, 'dataframe': dataframe} # no 'centroid' for the root cluster
left_child, right_child = bipartition(wiki_data, maxiter=100, num_runs=6, seed=1)

In [23]:


def display_single_tf_idf_cluster(cluster, map_index_to_word):
    wiki_subset   = cluster['dataframe']
    tf_idf_subset = cluster['matrix']
    centroid      = cluster['centroid']
    idx = centroid.argsort()[::-1]
    for i in xrange(5):
        print('{0:s}:{1:.3f}'.format(map_index_to_word['category'][idx[i]], centroid[idx[i]])),
    print('')
    distances = pairwise_distances(tf_idf_subset, [centroid], metric='euclidean').flatten()
    nearest_neighbors = distances.argsort()
    for i in xrange(8):
        text = ' '.join(wiki_subset[nearest_neighbors[i]]['text'].split(None, 25)[0:25])
        print('* {0:50s} {1:.5f}\n  {2:s}\n  {3:s}'.format(wiki_subset[nearest_neighbors[i]]['name'],
              distances[nearest_neighbors[i]], text[:90], text[90:180] if len(text) > 90 else ''))
    print('')


display_single_tf_idf_cluster(left_child, map_index_to_word)
display_single_tf_idf_cluster(right_child, map_index_to_word)

In [ ]:


# athletes = left_child
# non_athletes = right_child
# left_child_athletes, right_child_athletes = bipartition(athletes, maxiter=100, num_runs=6, seed=1)
# display_single_tf_idf_cluster(left_child_athletes, map_index_to_word)
# display_single_tf_idf_cluster(right_child_athletes, map_index_to_word)
# baseball = left_child_athletes
# ice_hockey_football = right_child_athletes
# left_child_ihs, right_child_ihs = bipartition(ice_hockey_football, maxiter=100, num_runs=6, seed=1)
# display_single_tf_idf_cluster(left_child_ihs, map_index_to_word)
# display_single_tf_idf_cluster(right_child_ihs, map_index_to_word)
# left_child_non_athletes, right_child_non_athletes = bipartition(non_athletes, maxiter=100, num_runs=6, seed=1)
# display_single_tf_idf_cluster(left_child_non_athletes, map_index_to_word)
# display_single_tf_idf_cluster(right_child_non_athletes, map_index_to_word)
# male_non_athletes = left_child_non_athletes
# female_non_athletes = right_child_non_athletes
# left_child_male_non_athletes, right_child_male_non_athletes = bipartition(male_non_athletes, maxiter=100, num_runs=6, seed=1)
# display_single_tf_idf_cluster(left_child_male_non_athletes, map_index_to_word)
# display_single_tf_idf_cluster(right_child_male_non_athletes, map_index_to_word)
# left_child_female_non_athletes, right_child_female_non_athletes = bipartition(female_non_athletes, maxiter=100, num_runs=6, seed=1)
# display_single_tf_idf_cluster(left_child_female_non_athletes, map_index_to_word)
# display_single_tf_idf_cluster(right_child_female_non_athletes, map_index_to_word)